5-ExamplePythonAPI_BalancingRobot.ipynb

    v0  2021-29-06  Mohamed Amghar et Alexis Boisseau 

# API Python BalancingRobot

This notebook imports 2 modules from the directory `coppeliasim/env` :
- sim.py
- simConst.py 

It also needs the appropriate remote API library: "remoteApi.dll" (Windows), "remoteApi.dylib" (Mac) or "remoteApi.so" (Linux)

The CoppeliaSim Python API is documented here: http://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctionsPython.htm

The cell bellow finds the relative path of the directory `coppeliasim_api/env` where the modules `sim` and `simConst` live:<br>
if not found in the current working dir it tries recursively to find a parent directory that holds `coppeliasim/env`.<br>
When found, it sets `root_dir` to this value and adds relevant paths to the list sys.path:

In [1]:
import sys, os

# initialize default path values:
target_dir = "coppeliasim_api/env"
root_dir = os.getcwd()
copsim_env_path = target_dir

if not os.path.isdir(target_dir):
    while not os.path.isdir(copsim_env_path):
        copsim_env_path = os.path.join('..', copsim_env_path)
    root_dir = copsim_env_path.replace(target_dir, "")

# run notebook in root dir and add the required paths to sys.path:
if  root_dir !=  os.getcwd():  
    os.chdir(root_dir)
    sys.path.append(root_dir)
    sys.path.append(target_dir)

print(f"\troot directory: <{root_dir}>")
print(f"\tworking directory is now: <{os.getcwd()}>")

# automatic reload of modules when modification
%load_ext autoreload
%autoreload 2

	root directory: <../../>
	working directory is now: </home/alexis/ENSEIRB/2A/StagePERSEUS/miniapterros>


In [2]:
try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported.')
    print ('--------------------------------------------------------------')
    print ('')
    raise

import time, sys
import numpy as np
from CopSim import Simulator

error_mess = 'Remote API error code: '
OK = sim.simx_return_ok

opmode_blocking    = sim.simx_opmode_blocking
opmode_oneshot     = sim.simx_opmode_oneshot
opmode_buffer      = sim.simx_opmode_buffer
opmode_streaming   = sim.simx_opmode_streaming
opmode_discontinue = sim.simx_opmode_discontinue

## >>> Automatic launch of a CoppeliaSim instance:

**Only for Linux** !!! <br>
=> Windows & Mac : run CoppeliaSim "by hand" and go to the cell "Connection to the CoppeliaSim simulator...<br><br>
The file `constants.py` in `coppeliasim_api/env` sets COPSIM_DIR to the name of the CoppeliaSim installation directory... modify it as needed.

In [3]:
import platform
if platform.system() =='Linux':
    # LINUX CUSTOMIZE : the file 'constants.py' from 'coppeliasim_api/env' sets COPSIM_DIR 
    # to the name of the CoppeliaSim installation directory : modify it as needed...
    from constants import COPSIM_DIR
    print(f"CoppeliaSim installation found in directory <{COPSIM_DIR}>")
    simulator = Simulator(19997, COPSIM_DIR, headless=False, verbose=1)
    simulator.start()
else:
    print("""Only for Linux platform !!!
Windows & Mac : run CoppeliaSim "by hand" and go to the cell "Connection to the CoppeliaSim simulator...""")    

CoppeliaSim installation found in directory <CoppeliaSim_Edu_V4_2_0_Ubuntu20_04>
[CopSim] starting an instance of CoppeliaSim...
[Instance] trying to start <./coppeliaSim.sh -gREMOTEAPISERVERSERVICE_19997_FALSE_TRUE -vnone>
[CopSim] CoppeliaSim instance started, remote API connection OK.


## Connection to the CoppeliaSim simulator:

In [4]:
sim.simxFinish(-1) # just in case, close all opened connections

clientID = sim.simxStart('127.0.0.1',19997,True,True,5000,5) # Connect to V-REP

if clientID == -1:
    print ('NOT Connected to remote API server')
    sys.exit("Could not connect")
else:
    print ('Successfully connected to the remote CoppeliaSim server with clientID={}'.format(clientID))

Successfully connected to the remote CoppeliaSim server with clientID=0


## API-Python : function simxLoadScene

## simxLoadScene

In [5]:
scene = "./balancing_robot/copsim/Balancing-robot.ttt"
res = sim.simxLoadScene(clientID, scene, 0xFF, opmode_blocking)
if res == OK:
    print ('Scene <{}> successfully loaded !'.format(scene))
else:
    print (error_mess, res)

Scene <./balancing_robot/copsim/Balancing-robot.ttt> successfully loaded !


### Obtenir les handles 

In [6]:
res, joint_r = sim.simxGetObjectHandle(clientID, "Joint_R", opmode_blocking)
if res != OK: print (error_mess, res)

res, joint_l = sim.simxGetObjectHandle(clientID, "Joint_L", opmode_blocking)
if res != OK: print (error_mess, res)

res, robot = sim.simxGetObjectHandle(clientID, "Planche4", opmode_blocking)
if res != OK: print (error_mess, res)

res, wheel_r = sim.simxGetObjectHandle(clientID, "Wheel_R", opmode_blocking)
if res != OK: print (error_mess, res)

res, wheel_l = sim.simxGetObjectHandle(clientID, "Wheel_L", opmode_blocking)
if res != OK: print (error_mess, res)

print (f'Handles:\n\tJoint_R: {joint_r}, Wheel_R: {wheel_r}')
print (f'\tJoint_L: {joint_l}, Wheel_L: {wheel_l}')
print (f'\tPlanche4: {robot}')

Handles:
	Joint_R: 15, Wheel_R: 18
	Joint_L: 16, Wheel_L: 17
	Planche4: 19


### Obtenir l'inclinaison du robot

In [7]:
res, orientation = sim.simxGetObjectOrientation(clientID, robot, -1, opmode_blocking)
if res == OK:
    print(f"Robot orientation (Euler angles (x,y,z)): {orientation} [rad]")
    print(f"Robot angle on x axis: {orientation[1]:5.2f} rad or {np.degrees(orientation[1]):5.2f} °")
else:
    print (error_mess, res)

Robot orientation (Euler angles (x,y,z)): [-5.632641114061698e-05, 1.0913936421275139e-11, -1.5707967281341553] [rad]
Robot angle on x axis:  0.00 rad or  0.00 °


### Modifier l'inclinaison du robot

In [8]:
target_orientation = [0.0, 1.0, 0.0]  # Euler angles in radian
sim.simxSetObjectOrientation(clientID, robot, -1, target_orientation, opmode_oneshot)

res, val_after = sim.simxGetObjectOrientation(clientID, robot, -1, opmode_blocking)
if res == OK:
    print(f"Robot orientation (Euler angles (x,y,z)): {val_after}")
    print(f"Robot angle on x axis: {val_after[1]:5.2f}")
else:
    print (error_mess, res)

Robot orientation (Euler angles (x,y,z)): [-0.0, 1.0, -0.0]
Robot angle on x axis:  1.00


## Obtenir l'état du robot (position, angles et leurs dérivées)

### Get X position:

In [9]:
res, pos = sim.simxGetObjectPosition(clientID, robot, -1, opmode_blocking) # x: robot position vector
if res == OK:
    print(f"Robot position vector [x,y,z]: {pos} [m]")
    print(f"Robot x position {pos[0]:6.3f} [m]")
else:
    print (error_mess, res)

Robot position vector [x,y,z]: [7.450580596923828e-09, -1.1920928955078125e-07, 0.11910025775432587] [m]
Robot x position  0.000 [m]


### Get theta angle:

In [10]:
res, orientation = sim.simxGetObjectOrientation(clientID, robot, -1, opmode_blocking)
if res == OK:
    print(f"Robot orientation (Euler angles)): {orientation} [rad]")
    print(f"Robot angle 'theta' : {orientation[1]:5.2f} rad or {np.degrees(orientation[1]):5.2f} °")
else:
    print (error_mess, res)

Robot orientation (Euler angles)): [-0.0, 1.0, -0.0] [rad]
Robot angle 'theta' :  1.00 rad or 57.30 °


### Get linear & angular velocity:

In [11]:
res, lin_vel, ang_vel = sim.simxGetObjectVelocity(clientID, robot, opmode_blocking) 
if res == OK:
    print(f"Robot velocity\n\tlinear : {lin_vel} [m/s],\n\tangular: {ang_vel} [rad/s]")
    print(f"Robot linear  velocity (X')    : {lin_vel[0]: 5.2f} [m/s]")
    print(f"Robot angular velocity (theta'): {lin_vel[0]: 5.2f} [rad/s]")
else:
    print (error_mess, res)

Robot velocity
	linear : [0.0, 0.0, 0.0] [m/s],
	angular: [0.0, 0.0, 0.0] [rad/s]
Robot linear  velocity (X')    :  0.00 [m/s]
Robot angular velocity (theta'):  0.00 [rad/s]


### Robot state vector $(x, x', \theta, \theta')$:

In [12]:
res, alpha = sim.simxGetJointPosition(clientID, joint_l, opmode_blocking) 
res, _, alpha_dot = sim.simxGetObjectVelocity(clientID, joint_l, opmode_blocking) 
res, orientation = sim.simxGetObjectOrientation(clientID, robot, -1, opmode_blocking)
res, lin_vel, ang_vel = sim.simxGetObjectVelocity(clientID, robot, opmode_blocking) 
R = 4.5e-2 #rayon de la roue
if res == OK:
    x = R*alpha
    x_dot = R*alpha_dot[2]
    theta = orientation[1]
    theta_dot = ang_vel[0]
    print(f"Robot state: ({x},{x_dot},{theta}, {theta_dot})")
else:
    print (error_mess, res)

Robot state: (0.0,0.0,1.0, 0.0)


### Fixer une vitesse de rotation des roues (via les deux joints)

In [15]:
sim.simxSetJointTargetVelocity(clientID, joint_l, 50, opmode_oneshot)
sim.simxSetJointTargetVelocity(clientID, joint_r, 50, opmode_oneshot)

0

## simxCloseScene

In [ ]:
res = sim.simxCloseScene(clientID, opmode_blocking)

 ## Close the simulator

 **Only for Linux** !!! <br>
=> Windows & Mac : close the CoppeliaSim window 'by hand'...

In [ ]:
if platform.system() =='Linux':
    simulator.end()
else:
    print("Windows & Mac : close the CoppeliaSim window 'by hand'...")